In [ ]:
%run "../library.py"

In [3]:
# prepare full training and test dataset
# http://stats.stackexchange.com/questions/82923/mixing-continuous-and-binary-data-with-linear-svm
train_data = preprocessing.scale(np.hstack([train_cnn,train_bow,train_attribs]), axis=0)
# train_data = np.hstack([train_data,train_attribs])
test_data = np.hstack([test_cnn,test_bow,test_attribs])
# test_data = np.hstack([test_data,test_attribs])

means = np.mean(train_data,axis=0)
stds = np.std(train_data,axis=0)
test_data = (test_data - means[None,:])/ stds[None,:]

In [15]:
#svm classification


svm_accuracy = []
predictions, svmmodel = 0, 0
for train, test in  KFold(len(train_data),n_folds=5):
    X_train,Y_train, X_test,Y_test = train_data[train],train_labels[train],train_data[test],train_labels[test]
    svmmodel = svm.SVC(kernel='linear', probability=True).fit(X_train,Y_train)
    score = svmmodel.score(X_test,Y_test)
    svm_accuracy.append(score)
    predictions = svmmodel.predict_proba(X_test)
    print score
    break
print svm_accuracy, "mean: ", np.mean(svm_accuracy)

0.398333333333
[0.39833333333333332] mean:  0.398333333333


In [21]:
'''
1. get predictions_cnn, predictions_bow and predictions_attribs set similar to the above
2. multiply the respective predictions array by the respective weights. 
eg. for cnn, the weight is 39/(39+12+27)
3. get a prediction_mean which averages the 3 prediction arrays
4. pick the maximum value and get the index for the maximum value
5. the svmmodel.classes_[max_index] will be the predicted class
REMEMBER: save the svm model which you fit. and do a single split. shouldn't use kfold
X_train, X_test, y_train, y_test = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)
'''
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)
print X_train.shape, X_test.shape, y_train.shape, y_test.shape

(2400, 8294) (600, 8294) (2400,) (600,)


In [62]:
# LDA
sklearn_lda = LDA(n_components=200)
train_data_lda = sklearn_lda.fit_transform(train_data, train_labels)

/Users/2359media/anaconda/lib/python2.7/site-packages/sklearn/lda.py:371: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


In [86]:
svm_accuracy = []
for train, test in  KFold(len(train_data_lda),n_folds=5):
    X_train,Y_train, X_test,Y_test = train_data_lda[train],train_labels[train],train_data_lda[test],train_labels[test]
    score = svm.SVC(kernel='linear').fit(X_train,Y_train).score(X_test,Y_test)
    svm_accuracy.append(score)
    print score
    
print svm_accuracy, "mean: ", np.mean(svm_accuracy)

1.0
1.0
1.0
1.0
1.0
[1.0, 1.0, 1.0, 1.0, 1.0] mean:  1.0


In [80]:
#scale test set

test_data_lda = sklearn_lda.transform(test_data)
predict = svm.SVC(kernel='linear').fit(train_data_lda,train_labels).predict(test_data_lda)
kaggle_output(predict)

[['0000.jpg' 'soccer_field']
 ['0001.jpg' 'city']
 ['0002.jpg' 'recreation_room']
 ..., 
 ['0997.jpg' 'fastfood_restaurant']
 ['0998.jpg' 'dorm_room']
 ['0999.jpg' 'pasture']]


In [11]:
# PCA + Adaboost


# train_data_pca = get_pca(train_data, 2)
# test_data_pca = get_pca(test_data, 200)
accuracies = []
for train, test in  KFold(len(train_data),n_folds=3):
    clf = AdaBoostClassifier(svm.SVC(probability=True,kernel='linear'),n_estimators=5)
    X_train,Y_train, X_test,Y_test = train_data[train],train_labels[train],train_data[test],train_labels[test]
    score = clf.fit(X_train,Y_train).score(X_test,Y_test)
    accuracies.append(score)
    print score


0.325
0.308
0.322


In [9]:
# tdpca = get_pca(train_data, 2)
# clf = AdaBoostClassifier(svm.SVC(probability=True,kernel='linear'),n_estimators=5)
clf.fit(tdpca,train_labels)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='linear', max_iter=-1, probability=True, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
          learning_rate=1.0, n_estimators=5, random_state=None)

In [10]:
clf.predict(get_pca(test_data, 2))

array(['valley', 'forest_road', 'dentists_office', 'city', 'city',
       'fishpond', 'dentists_office', 'bookstore', 'fishpond', 'fishpond',
       'coast', 'valley', 'city', 'coast', 'ocean', 'valley', 'coast',
       'fishpond', 'bookstore', 'television_studio', 'valley',
       'television_studio', 'valley', 'bookstore', 'sea_cliff',
       'dentists_office', 'bookstore', 'bookstore', 'television_studio',
       'newsroom', 'television_studio', 'television_studio', 'vineyard',
       'sea_cliff', 'bookstore', 'coast', 'bookstore', 'valley',
       'television_studio', 'television_studio', 'valley', 'bar', 'city',
       'city', 'valley', 'city', 'valley', 'city', 'television_studio',
       'bar', 'bookstore', 'television_studio', 'dentists_office', 'city',
       'valley', 'valley', 'newsroom', 'valley', 'forest_road', 'valley',
       'dining_car', 'city', 'bookstore', 'valley', 'television_studio',
       'fishpond', 'bookstore', 'fishpond', 'bar', 'television_studio',
       'v